In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=5bced0f6f73be84fa6d9879e0f77f2d02c68885c802a6750d87ac2936099702f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
!wget https://jpbarddal.github.io/assets/data/bigdata/transactions_amostra.csv.zip
!unzip transactions_amostra.csv.zip

--2023-05-29 22:08:17--  https://jpbarddal.github.io/assets/data/bigdata/transactions_amostra.csv.zip
Resolving jpbarddal.github.io (jpbarddal.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to jpbarddal.github.io (jpbarddal.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47513871 (45M) [application/zip]
Saving to: ‘transactions_amostra.csv.zip’

transactions_amostr 100%[===================>]  45.31M   189MB/s    in 0.2s    

2023-05-29 22:08:17 (189 MB/s) - ‘transactions_amostra.csv.zip’ saved [47513871/47513871]

Archive:  transactions_amostra.csv.zip
  inflating: transactions_amostra.csv  
  inflating: __MACOSX/._transactions_amostra.csv  


In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark import SparkFiles
#from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.master('local').appName('tdeSpark').getOrCreate()
sc = spark.sparkContext

In [ ]:
df = sc.textFile("transactions_amostra.csv")

In [ ]:
df.take(5)

['country_or_area;year;comm_code;commodity;flow;trade_usd;weight_kg;quantity_name;quantity;category',
 'Belgium;2016;920510;Brass-wind instruments;Export;571297;3966.0;Number of items;4135.0;92_musical_instruments_parts_and_accessories',
 'Guatemala;2008;660200;Walking-sticks, seat-sticks, whips, etc.;Export;35022;5575.0;Number of items;10089.0;66_umbrellas_walking_sticks_seat_sticks_whips_etc',
 'Barbados;2006;220210;Beverage waters, sweetened or flavoured;Re-Export;81058;44458.0;Volume in litres;24113.0;22_beverages_spirits_and_vinegar',
 'Tunisia;2016;780411;Lead foil of a thickness <2mm;Import;4658;121.0;Weight in kilograms;121.0;78_lead_and_articles_thereof']

In [ ]:
# ======== QUESTÃO 1 ========

# FILTRA AS TRNSAÇÕES ENVOLVENDO O BRASIL
transacoesBrasil = df.filter(lambda linha: "Brazil" in linha)

# CONVERTE PARA CHAVE
chaveQuestao1 = transacoesBrasil.map(lambda linha: ("brazil", 1))

# CALCULA O TOTAL DE TRANSAÇÕES
totalDeTransacoesBrasil = chaveQuestao1.reduceByKey(lambda x, y: x + y)

contador = totalDeTransacoesBrasil.collect()[0][1]
print(contador)


27693


In [ ]:
# ======== QUESTÃO 2 ========

# DEFINE A CHAVE NA COMBINAÇÃO DO TIPO DE FLOW E ANO, E JA FAZ O REDUCE SOMANDO O VALORES
transacoesPorFlowTipoAno = df.map(lambda linha: ((linha.split(";")[4], linha.split(";")[1]), 1)).reduceByKey(lambda x, y: x + y)
print("TIPO DE FLOW, ANO \t TRANSAÇÕES")

# ITERA SOBRE OS VALORES E IMPRIME O TIPO DE FLOW, ANO E A QUANTIDADE
for tipoAno, contador in transacoesPorFlowTipoAno.collect():
    print( tipoAno, "\t", contador)
     

TIPO DE FLOW, ANO 	 TRANSAÇÕES
('Import', '2013') 	 30689
('Import', '2010') 	 31435
('Import', '2003') 	 29997
('Export', '2015') 	 17756
('Export', '2014') 	 17984
('Import', '2012') 	 29987
('Import', '2014') 	 29723
('Import', '1995') 	 19375
('Export', '2012') 	 17863
('Export', '2013') 	 18063
('Re-Export', '2003') 	 1706
('Re-Export', '2013') 	 2225
('Re-Import', '2005') 	 1323
('Re-Export', '1995') 	 1132
('Re-Export', '1990') 	 518
('flow', 'year') 	 1
('Export', '2008') 	 17445
('Import', '2008') 	 29883
('Import', '2005') 	 31124
('Export', '2009') 	 17825
('Export', '1989') 	 3547
('Re-Import', '2011') 	 1456
('Re-Import', '2012') 	 1418
('Re-Import', '2014') 	 1457
('Re-Import', '2003') 	 1242
('Re-Export', '1992') 	 764
('Export', '2016') 	 15260
('Export', '2011') 	 17706
('Import', '1994') 	 15718
('Export', '2004') 	 17944
('Export', '2003') 	 17484
('Import', '2015') 	 28834
('Import', '1990') 	 4866
('Export', '1993') 	 7766
('Import', '2002') 	 28956
('Export', '199

In [ ]:
# ======== QUESTÃO 3 ========

 # FILTRA O VALOR DAS TRANSAÇÕES E O ANO E TIRA OS PONTOS PRESENTES
valorCommodities = df.filter(lambda linha: linha.split(";")[5].replace(".", "").isdigit()).map(lambda linha: (linha.split(";")[1], float(linha.split(";")[5])))

# AGREGAS OS VALORES DAS COMODITEIS POR ANO EM UMA TUPLA COM A SOMA TOTAL E QUANTIDADE DE OCORRÊNCIAS
valorCommoditiesPorAno = valorCommodities.aggregateByKey((0.0, 0),
                                                                  lambda x, y: (x[0] + y, x[1] + 1),
                                                                  lambda a, b: (a[0] + b[0], a[1] + b[1]))

# FAZ A MÉDIA
mediaPorAno = valorCommoditiesPorAno.mapValues(lambda x: x[0] / x[1])

# PRINTA O ANO E VALOR MÉDIO
print("ANO \t VALOR MÉDIO")
for ano, valorMedio in mediaPorAno.collect():
    print(ano, "\t", valorMedio)

ANO 	 VALOR MÉDIO
2016 	 29418327.57526777
2011 	 33559943.50890797
2003 	 13028917.611334749
1993 	 10353959.855309162
2007 	 23710673.174875777
2014 	 46120404.41345007
2012 	 39028921.881444596
1995 	 12286454.103356835
2006 	 21175872.541099638
1997 	 9549881.214776853
2005 	 18673099.052178193
1999 	 9561516.927263891
1992 	 9402960.863025468
1994 	 11350325.049077941
2004 	 15388487.793083541
1991 	 13069223.85515173
2002 	 12653310.252481185
2008 	 33285215.353921242
2001 	 9942220.288239626
2013 	 33061151.28882995
2010 	 26909386.074333776
2015 	 31115574.884196
1990 	 11724265.86778952
1996 	 11945524.161286663
1998 	 10175610.459598826
2009 	 25068409.504465386
2000 	 13547613.067035558
1988 	 18642970.55638571
1989 	 11263871.329920229


In [ ]:
# ======== QUESTÃO 4 ========

# FILTRA AS LINHAS CUJO O PAIS SEJA O BRASIL E O TIPO DE FLOW SEJA A EXPORTAÇÃO
exportBrasil = df.filter(lambda x: "Brazil" in x and x.split(';')[4] == "Export")

# CRIA A CHAVE QUE POSSUI OS VALORES DE ANO, TIPO DE UNIDADE, COMODITIE E QUANTIDADE
precoPorTipoDeUnidade = exportBrasil.map(lambda x: ((x.split(';')[1], x.split(';')[7], x.split(';')[3], x.split(';')[8]), float(x.split(';')[5])))

# AGREGA E FAZ A SOMA DE TODOS QUE POSSUEM O MESMO VALORES
soma = precoPorTipoDeUnidade.aggregateByKey((0.0, 0), lambda x,y: (x[0] + y, x[1] + 1), lambda x,y: (x[0] + y[0], x[1] + y[1]))

# CALCULA A MÉDIA
valorMedio = soma.mapValues(lambda x: x[0]/x[1])

# ITERA E PRINTA AS 5 PRIMEIRAS LINHAS
print("ANO \t   TIPO UNIDADE \t\t\t    COMMODITIE \t QUANTIDADE \t VALOR MÉDIO")
contador = 0
for media in valorMedio.collect():
    if contador < 5:
        print(media)
        contador += 1
    else:
        break


In [ ]:
# ======== QUESTÃO 5 ========

# FILTRANDO A BASE DE DADOS PELO TIPO DE UNIDADE, ANO E PRECO (MIN, MAX E MEDIA)
transacoes = df.map(lambda linha: ((linha.split(";")[7], linha.split(";")[1]),
                                   (float(linha.split(";")[5]), float(linha.split(";")[5]), float(linha.split(";")[5]), 1) if linha.split(";")[5].isdigit() else (0.0, 0.0, 0.0, 0.0)))

# ENCONTRANDO MINIMO, O MAXIMO, SOMA DO PRECOS E A SOMA DAS QUANTIDADES
somas = transacoes.reduceByKey(lambda x, y: (min(x[0], y[0]), max(x[1], y[1]), x[2] + y[2], x[3] + y[3]))

# MAPEANDO A BASE DE DADOS E ENCONTRANDO O MIN E MAX E FAZENDO A MEDIA
mediaPorTipoAno = somas.mapValues(lambda v: (v[0], v[1], v[2] / v[3]))
mediaPorTipoAno.take(5)




In [ ]:
# ======== QUESTÃO 6 ========

# FILTRA A BASE DE DADOS QUANDO O TIPO DE FLOW FOR A EXPORTAÇÃO
export = df.filter(lambda linha: linha.split(";")[4] == "Export")

# MAPEIA CADA LINHA REFERENTE A CHAVE PAÍS E VALOR DO COMODITIE
precoPorPais = export.map(lambda linha: (linha.split(";")[0], float(linha.split(";")[5])))

# FAZ A SOMA E CALCULA OS VALORES PARA CADA PAÍS
soma = precoPorPais.aggregateByKey(
    (0.0, 0),
    lambda x, y: (x[0] + y, x[1] + 1),
    lambda x1, x2: (x1[0] + x2[0], x1[1] + x2[1])
)

# CALCULA O VALOR MÉDIO PARA CADA PAÍS
mediaPorPais = soma.mapValues(lambda x: x[0] / x[1])

# PROCURA O PAÍS COM MAIS MÉDIA
paisMaiorMedia = mediaPorPais.max(lambda a: a[1])

# IMPRIME UM ÚNICO PAÍS NO OUTPUT
print("PAÍS COM MAIOR MÉDIA NO TIPO EXPORT \n", paisMaiorMedia[0], paisMaiorMedia[1])

País com o maior preço médio de commodities na categoria Exportação 
 Angola 16369666068.142857


In [ ]:
# ======== QUESTÃO 7 ========
# FILTRA AS LINHAS PARA AQUELAS QUE SÃO DO ANO DE 2016
ano = df.filter(lambda linha: linha.split(';')[1] == "2016")

# MAPEIA CADA LINHA, ONDE A CHAVE É O COMMODITIE E SEU VALOR É UMA TUPLA DO TIPO DE FLOW E QUANTIDADE
comoditieTipoQuantidade = ano.map(lambda linha: (linha.split(';')[3], (linha.split(';')[4], float(linha.split(';')[8]))))

# FILTRA AS LINHAS QUE SÃO DO TIPO RE-EXPORT
reExport = comoditieTipoQuantidade.filter(lambda x: x[1][0] == "Re-Export")
reExport = reExport.map(lambda x: (x[0], x[1][1]))
# FAZ A SOMA
somaReExport = reExport.reduceByKey(lambda x,y: x+y)
# PEGA O VALOR MÁXIMO DE COMODITIES
maxReExport = somaReExport.max(lambda x: x[1])

# FILTRA AS LINHA QUE SÃO DO TIPO RE-IMPORT
reImport = comoditieTipoQuantidade.filter(lambda x: x[1][0] == "Re-Import")
reImport = reImport.map(lambda x: (x[0], x[1][1]))
# FAZ A SOMA
somaReImport = reImport.reduceByKey(lambda x,y: x + y)
# PEGA O VALOR MÁXIMO DE COMODITIES
maxReImport = somaReImport.max(lambda x: x[1])

# FILTRA AS LINHAS QUE SÃO DO TIPO EXPORT
export = comoditieTipoQuantidade.filter(lambda x: x[1][0] == "Export")
export = export.map(lambda x: (x[0], x[1][1]))
# FAZ A SOMA
somaExport = export.reduceByKey(lambda x,y: x+y)
# PEGA O VALOR MÁXIMO DE COMODITIES
maxExport = somaExport.max(lambda x: x[1])

# FILTRA AS LINHAS QUE SÃO DO TIPO IMPORT
importt = comoditieTipoQuantidade.filter(lambda x: x[1][0] == "Import")
importt = importt.map(lambda x: (x[0], x[1][1]))
# FAZ A SOMA
somaImport = importt.reduceByKey(lambda x,y: x+y)
# PEGA O VALOR MÁXIMO DE COMODITIES
maxImport = somaImport.max(lambda x: x[1])

#PRINTA O TIPO DE FLOW, O COMMODITIE MAIS COMERCIALIZADO E A QUANTIDADE
print("FLOW \t   QUANTIDADE")
print("Re-Export:", maxReExport)
print("Re-Import:", maxReImport)
print("Export:", maxExport)
print("Import:", maxImport)



FLOW 	   QUANTIDADE
Re-Export: ('Safety razor blades, including blanks in strips', 1261968000.0)
Re-Import: ('Chem wood pulp, soda/sulphate, non-conifer, bleached', 38774873.0)
Export: ('Iron ore, concentrate, not iron pyrites,unagglomerate', 379546246752.0)
Import: ('Petroleum oils, oils from bituminous minerals, crude', 258289373308.0)
